# importing data from database

In [1]:
import numpy as np
import pandas as pd
from SportsExperiencePlatform.data import connect_db, get_data
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import save_npz, load_npz

In [2]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [3]:
conn = connect_db()

if conn:
    users_df, events_df, ahoy_events_df = get_data(conn)

/Users/shreetamakarmakar/.pyenv/versions/3.8.13/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/shreetamakarmakar/.pyenv/versions/3.8.13/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/shreetamakarmakar/.pyenv/versions/3.8.13/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlche

In [4]:
users_df.shape, events_df.shape, ahoy_events_df.shape

((11, 16), (882, 17), (31, 6))

In [5]:
users_df.columns

Index(['id', 'email', 'encrypted_password', 'reset_password_token',
       'reset_password_sent_at', 'remember_created_at', 'created_at',
       'updated_at', 'first_name', 'last_name', 'description', 'date_of_birth',
       'gender', 'location', 'latitude', 'longitude'],
      dtype='object')

In [6]:
user_loc_df = users_df[['location','latitude','longitude']]
user_loc_df.head(15)

,location,latitude,longitude
0,Munich,48.135124,11.581981
1,Berlin,52.520008,13.404954
2,Berlin,52.520008,13.404954
3,Hamburg,53.551086,9.993682
4,Cologne,50.937531,6.960279
5,Cologne,50.937531,6.960279
6,Stuttgart,48.775845,9.182932
7,None,NaN,NaN
8,None,NaN,NaN
9,Hamburg,53.551086,9.993682


In [7]:
users_df.at[8, 'location'] = 'Berlin'

In [8]:
users_df.at[8, 'latitude'] = 52.520008 
users_df.at[8, 'longitude'] = 13.404954
users_df.at[11, 'location'] = 'Berlin'
users_df.at[11, 'location'] = 'Berlin'
users_df.at[11, 'latitude'] = 52.520008 
users_df.at[11, 'longitude'] = 13.404954
users_df.at[10, 'location'] = 'Berlin'
users_df.at[10, 'latitude'] = 52.520008 
users_df.at[10, 'longitude'] = 13.404954

In [9]:
users_df.head(15)

,id,email,encrypted_password,reset_password_token,reset_password_sent_at,remember_created_at,created_at,updated_at,first_name,last_name,description,date_of_birth,gender,location,latitude,longitude
0,55.0,johanna@gmail.com,$2a$12$CqtujmUxnctn4PYh/LQ7Yen4N9zGBa8mAkW5lJk...,None,None,NaT,2022-05-28 12:26:38.267750,2022-05-28 12:26:38.283275,Johanna,Maier,"Hi, my name is Johanna! I want to go on my nex...",1993-01-25,female,Munich,48.135124,11.581981
1,56.0,anjali@gmail.com,$2a$12$zoaJySL5quI21u316oCrzuwLap5n3iaoJoFEt8O...,None,None,NaT,2022-05-28 12:26:39.191914,2022-05-28 12:26:39.203232,Anjali,Kumar,"Hi, my name is Anjali! I want to go on my next...",1993-01-25,female,Berlin,52.520008,13.404954
2,57.0,alexandra@gmail.com,$2a$12$EFJExS3e79KnS1Sj5SU7iuA0oE4Fe3cghnIcWCj...,None,None,NaT,2022-05-28 12:26:40.648549,2022-05-28 12:26:40.656142,Alexandra,Stroe,"Hi, my name is Alexandra! I want to go on my n...",1993-01-25,female,Berlin,52.520008,13.404954
3,58.0,ieva@gmail.com,$2a$12$2KfdqVXZIU7RbiktQW1GAuaBnIcIfHBSc68kwWc...,None,None,NaT,2022-05-28 12:26:41.476255,2022-05-28 12:26:41.484819,Ieva,Jirgensone,"Hi, my name is Ieva! I want to go on my next s...",1993-01-25,female,Hamburg,53.551086,9.993682
4,61.0,shreetama@gmail.com,$2a$12$Y3BZQdV7MhHLkHWNjXNI2.0/xszGFofvoLFtvwq...,None,None,NaT,2022-05-28 12:26:43.898237,2022-05-28 12:26:43.910509,Shreetama,Karmakar,"Hi, my name is Shreetama! I want to go on my n...",1993-01-25,female,Cologne,50.937531,6.960279
5,62.0,andrew@gmail.com,$2a$12$WQNQrH8cu5Pmvk228dOzCONyjixPUEhV.snxt1j...,None,None,NaT,2022-05-28 12:26:44.867437,2022-05-28 12:26:44.878280,Andrew,Erlanger,"Hi, my name is Andrew! I want to go on my next...",1993-01-25,male,Cologne,50.937531,6.960279
6,63.0,killian@gmail.com,$2a$12$EGWPW0BWsJk07er5uGiXjeNaLFqubjkIvSaq6VS...,None,None,NaT,2022-05-28 12:26:45.774583,2022-05-28 12:26:45.784615,Killian,Dectot,"Hi, my name is Killian! I want to go on my nex...",1993-01-25,male,Stuttgart,48.775845,9.182932
7,64.0,shreetama1985@gmail.com,$2a$12$8O0AVmviqFX4G.eP/ual6uGUfv6wiq3C5Y.5VcS...,None,None,NaT,2022-05-28 13:37:08.002699,2022-05-28 13:37:08.002699,Shreetama,Karmakar,None,None,None,None,NaN,NaN
8,65.0,rpadev7@yahoo.com,$2a$12$MdWAEIjH9EyMqA6dg5Xz8e5DuUlOkGN.nR2PH9d...,None,None,NaT,2022-06-04 08:04:14.868248,2022-06-04 08:04:14.868248,Julian,Mühlbauer,None,None,None,Berlin,52.520008,13.404954
9,60.0,julian@gmail.com,$2a$12$JaT8SCjL4GJ4gokRU/GO5.1YpDCt00/GwNj8ChD...,None,None,2022-06-04 15:51:57.870746,2022-05-28 12:26:43.051327,2022-06-04 15:51:57.871148,Julian,Muhlbauer,"Hi, my name is Julian! I want to go on my next...",1993-01-25,male,Hamburg,53.551086,9.993682


In [10]:
ahoy_events_df.shape

(31, 6)

In [11]:
ahoy_events_df.user_id.unique()

array([64, 55, 65, 60, 56, 59])

In [12]:
ahoy_events_df.head()

,id,visit_id,user_id,name,properties,time
0,12,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:01.802118
1,13,51,64,View Offer Page ID 1804,"{'user': 64, 'offer': 1804}",2022-05-28 13:38:18.486753
2,14,51,64,View Offer Page ID 1676,"{'user': 64, 'offer': 1676}",2022-05-28 13:38:44.105830
3,15,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:54.335809
4,16,51,64,View Offer Page ID 1443,"{'user': 64, 'offer': 1443}",2022-05-28 13:39:00.433517


# simple tfidf model: using the raw description

## advanced cleaning of data

In [13]:
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
from googletrans import Translator

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shreetamakarmakar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:

# def to_english(string):
    

# # result.src
# if translator.detect(string).lang != 'en':
#     result = translator.translate(string)
#     result.text

In [16]:
from googletrans import Translator
def language_translation(string):
    
    translator = Translator()
    if translator.detect(string).lang != 'en':
        result = translator.translate(string)
        return result.text
    else:
        return string

In [17]:
language_translation('yo no sabe')

'I do not know'

In [18]:
ahoy_events_df.head()

,id,visit_id,user_id,name,properties,time
0,12,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:01.802118
1,13,51,64,View Offer Page ID 1804,"{'user': 64, 'offer': 1804}",2022-05-28 13:38:18.486753
2,14,51,64,View Offer Page ID 1676,"{'user': 64, 'offer': 1676}",2022-05-28 13:38:44.105830
3,15,51,64,View Offer Page ID 1841,"{'user': 64, 'offer': 1841}",2022-05-28 13:38:54.335809
4,16,51,64,View Offer Page ID 1443,"{'user': 64, 'offer': 1443}",2022-05-28 13:39:00.433517


In [19]:
users_df.head().T

,0,1,2,3,4
id,55.0,56.0,57.0,58.0,61.0
email,johanna@gmail.com,anjali@gmail.com,alexandra@gmail.com,ieva@gmail.com,shreetama@gmail.com
encrypted_password,$2a$12$CqtujmUxnctn4PYh/LQ7Yen4N9zGBa8mAkW5lJk...,$2a$12$zoaJySL5quI21u316oCrzuwLap5n3iaoJoFEt8O...,$2a$12$EFJExS3e79KnS1Sj5SU7iuA0oE4Fe3cghnIcWCj...,$2a$12$2KfdqVXZIU7RbiktQW1GAuaBnIcIfHBSc68kwWc...,$2a$12$Y3BZQdV7MhHLkHWNjXNI2.0/xszGFofvoLFtvwq...
reset_password_token,None,None,None,None,None
reset_password_sent_at,None,None,None,None,None
remember_created_at,NaT,NaT,NaT,NaT,NaT
created_at,2022-05-28 12:26:38.267750,2022-05-28 12:26:39.191914,2022-05-28 12:26:40.648549,2022-05-28 12:26:41.476255,2022-05-28 12:26:43.898237
updated_at,2022-05-28 12:26:38.283275,2022-05-28 12:26:39.203232,2022-05-28 12:26:40.656142,2022-05-28 12:26:41.484819,2022-05-28 12:26:43.910509
first_name,Johanna,Anjali,Alexandra,Ieva,Shreetama
last_name,Maier,Kumar,Stroe,Jirgensone,Karmakar


In [27]:
events_df.title

0                                 Gilde Bowling Wandsbek
1                                 Gilde Bowling Wandsbek
2                                           Bowlingabend
3                         Return To Love Virtual Retreat
4      Gaelic Football Training Hamburg m/f/d - all w...
                             ...                        
877                   Online: Meditation - Zeit für Dich
878                  Online: Meditate to regenerate (EN)
879                   Online: Meditation - Zeit für Dich
880    Sahaja Yoga Meditation für Anfänger und Fortge...
881    Sahaja Yoga Meditation für Anfänger und Fortge...
Name: title, Length: 882, dtype: object

In [28]:
# events_df['translated_title'] = events_df['title'].apply(lambda x: language_translation(x))
# events_df['translated_description'] = events_df['description'].apply(lambda x: language_translation(x))

ReadError: [Errno 54] Connection reset by peer

In [29]:
8# load spacy
nlp = spacy.load('en_core_web_sm')

def clean_string(text,stem="None"):

    final_string = ""

    # make lower
    text = text.lower()
    
    # remove http
    text = re.sub(r"http.*\.[a-z]{2,3}","",text)

    # remove www
    text = re.sub(r"www.*\.[a-z]{2,3}","",text)
    
    # remove line breaks
    text = re.sub(r'\n','',text)

    # remove puncuation
    translator = str.maketrans('','',string.punctuation)
    text = text.translate(translator)

    # remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    text_filtered = [word for word in text if not word in useless_words]

    # remove numbers
    text_filtered = [re.sub(r'\w*\d\w*','',w) for w in text_filtered]

    # stem or lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string
        
        

In [ ]:
# events_df['clean_description'] = events_df['translated_description'].apply(lambda x: clean_string(x))
# events_df['clean_title'] = events_df['translated_title'].apply(lambda x: clean_string(x))

In [30]:
events_df['clean_description'] = events_df['description'].apply(lambda x: clean_string(x))
events_df['clean_title'] = events_df['title'].apply(lambda x: clean_string(x))

In [ ]:
# events_df['translated_title'] = events_df['title'].apply(lambda x: language_translation(x))
# events_df['translated_description'] = events_df['description'].apply(lambda x: language_translation(x))

In [31]:
ahoy_events_df.user_id.unique()

array([64, 55, 65, 60, 56, 59])

In [32]:
users_df.head()

,id,email,encrypted_password,reset_password_token,reset_password_sent_at,remember_created_at,created_at,updated_at,first_name,last_name,description,date_of_birth,gender,location,latitude,longitude
0,55.0,johanna@gmail.com,$2a$12$CqtujmUxnctn4PYh/LQ7Yen4N9zGBa8mAkW5lJk...,None,None,NaT,2022-05-28 12:26:38.267750,2022-05-28 12:26:38.283275,Johanna,Maier,"Hi, my name is Johanna! I want to go on my nex...",1993-01-25,female,Munich,48.135124,11.581981
1,56.0,anjali@gmail.com,$2a$12$zoaJySL5quI21u316oCrzuwLap5n3iaoJoFEt8O...,None,None,NaT,2022-05-28 12:26:39.191914,2022-05-28 12:26:39.203232,Anjali,Kumar,"Hi, my name is Anjali! I want to go on my next...",1993-01-25,female,Berlin,52.520008,13.404954
2,57.0,alexandra@gmail.com,$2a$12$EFJExS3e79KnS1Sj5SU7iuA0oE4Fe3cghnIcWCj...,None,None,NaT,2022-05-28 12:26:40.648549,2022-05-28 12:26:40.656142,Alexandra,Stroe,"Hi, my name is Alexandra! I want to go on my n...",1993-01-25,female,Berlin,52.520008,13.404954
3,58.0,ieva@gmail.com,$2a$12$2KfdqVXZIU7RbiktQW1GAuaBnIcIfHBSc68kwWc...,None,None,NaT,2022-05-28 12:26:41.476255,2022-05-28 12:26:41.484819,Ieva,Jirgensone,"Hi, my name is Ieva! I want to go on my next s...",1993-01-25,female,Hamburg,53.551086,9.993682
4,61.0,shreetama@gmail.com,$2a$12$Y3BZQdV7MhHLkHWNjXNI2.0/xszGFofvoLFtvwq...,None,None,NaT,2022-05-28 12:26:43.898237,2022-05-28 12:26:43.910509,Shreetama,Karmakar,"Hi, my name is Shreetama! I want to go on my n...",1993-01-25,female,Cologne,50.937531,6.960279


In [33]:
events_df.shape[0]

882

In [34]:
events_df['description'][190]

"On this hike you can join a small group of passionate campers and hikers from this event: [https://www.meetup.com/BerlinTravel-Meetup/events/286074116/](https://www.meetup.com/BerlinTravel-Meetup/events/286074116/). On our way, there will be several swimming opportunities - fingers crossed that the weather will be good enough!\n\n**Tour highlights**\n\n* Lake Großer Wentowsee\n* Many little artificial lakes\n* [Ziegeleipark Mildenberg](https://www.ziegeleipark.de/)\n* Complete route: [View in browser](https://t1p.de/czsch) & [download as GPX/KML](https://easyupload.io/m/ohtsck)\n\n**How to get there**\n\n* Take the RE5 with direction Neustrelitz, arrival at Dannenwalde (bei Gransee) will be at 11:07.\n* We'll meet on the platform.\n* I recommend to buy a 9€ ticket which will be valid for the entire month in all of Germany (except intercity trains like ICE, IC, ...). If you already have an Abo for Berlin AB/ABC/whatever, this will count as a valid ticket as well.\n\n**Please bring**\n\

In [35]:
events_df['clean_description'][190]

'hike join small group passionate campers hikers event  way several swimming opportunities fingers crossed weather good enoughtour highlights lake großer wentowsee many little artificial lakes ziegeleipark mildenberg complete route view browsermohtsckhow get take  direction neustrelitz arrival dannenwalde bei gransee  well meet platform recommend buy € ticket valid entire month germany except intercity trains like ice ic already abo berlin ababcwhatever count valid ticket wellplease bring something eat drink swimming equipment'

#  recommender combining both title and description

## mashup of title and description

In [36]:
import random
'''this may be a bit dangerous but doing just for fun!!!'''
def random_generate_list(x):
    names = list(x.split())
    random.shuffle(names)
    if len(names) > 5:
        names = names[:5]
            
        
    return names

def random_generate_long_list(x):
    names = list(x.split())
    random.shuffle(names)
    if len(names) > 200:
        names = names[:200]
            
        
    return names

In [ ]:
events_df['random_title'] = events_df['clean_title'].apply(lambda x: random_generate_list(x))
events_df['random_description'] = events_df['clean_description'].apply(lambda x: random_generate_long_list(x))

In [ ]:
(' '.join(events_df["random_title"][0]) + ' ' + ' '.join(events_df["random_description"][190])).split()

In [ ]:
import random
def create_soup(x):
    soup = (" ".join(x["random_title"]) + " " + " ".join(x["random_description"])).split()
    random.shuffle(soup)
    return ' '.join(soup)
# df["soup"] = df.apply(create_soup, axis = 1)
# df["soup"].head(3)

In [ ]:
events_df['soup'] = events_df.apply(create_soup, axis = 1)

In [ ]:
events_df.soup[190]

## tfidf on clean data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf = TfidfVectorizer(stop_words="english")

In [ ]:
tfidf_matrix = tfidf.fit_transform(events_df["soup"])
tfidf_matrix.shape

In [ ]:
cosine_sim = tfidf_matrix.dot(tfidf_matrix.T)

In [ ]:
ahoy_events_df.head()

In [ ]:
title_to_index = pd.Series(events_df.index, index = events_df["title"]).drop_duplicates()

In [ ]:
ahoy_events_df.user_id.unique()

In [ ]:
# event_idx = ahoy_events_df[ahoy_events_df['user_id']==46].iloc[0].properties['offer']
# # now we get the title of the event
# title = events_df[events_df['id']==event_idx].title

In [ ]:
events_df.head()

In [ ]:
import datetime

In [ ]:
from datetime import date

def get_title(user_id, df = ahoy_events_df):
    ''' this finction grabs the title of an event used by an user from ahoy_events dataset'''
    
    event_idx = ahoy_events_df[ahoy_events_df['user_id']== user_id].iloc[0].properties['offer']
    # now we get the title of the event
    title = events_df[events_df['id']==event_idx].title
    
    return title

# def get_current_date():
    
    
    
# def get_user_loc():   

def content_recommender(user_id, cosine_sim = cosine_sim, df = events_df, title_to_index = title_to_index):
    '''This recommender finds out 20 closest matches with a given event'''
    
    #calling user id from ahoy_events
    
    title = get_title(user_id, df = ahoy_events_df)
    
    idx = title_to_index[title]
    
    sim_scores = list(enumerate(cosine_sim[idx].todense().tolist()[0]))
    # list of set [(index, sim score), (index, sim score), ...]
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Sort the list of set by sim score, index with highest simlirarity
    # Will be at the beginning of the list
    
    sim_scores = sim_scores[1:101] # top 100 events (first one is our input events so we ignore it)
    
    sport_indices = [i[0] for i in sim_scores] # we grab the indices of those 10 events
    
    
    return df['title'].iloc[sport_indices]

In [ ]:
ahoy_events_df.user_id.unique()

In [ ]:
content_recommender(65)

In [ ]:
#content_recommender("Along the shore of the Elbe to Aumühle (52 km/ hard)")

In [ ]:
type(events_df.latitude[0])

In [ ]:
def haversine_vectorized(user_lat,
                         user_lon,
                         event_lat,
                         event_lon):
    """
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df
        Computes distance in kms
    """

    lat_1_rad, lon_1_rad = np.radians(user_lat), np.radians(user_lon)
    lat_2_rad, lon_2_rad = np.radians(event_lat), np.radians(event_lon)
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [ ]:
haversine_vectorized(48.135124, 11.581981, 47.135124, 12.581981)

In [ ]:
users_df.id.unique()

In [ ]:
get_user_loc(65, df = users_df)

#  recommemder filter for future dates and nearest places 

In [ ]:
ahoy_events_df.head(15)

In [ ]:
events_df.head()

In [ ]:
from datetime import date

def get_title(user_id, df = ahoy_events_df):
    ''' this finction grabs the title of an event used by an user from ahoy_events dataset'''
    
    event_idx = ahoy_events_df[ahoy_events_df['user_id']== user_id].iloc[0].properties['offer']
    # now we get the title of the event
    title = events_df[events_df['id']==event_idx].title
    
    return title  

def get_user_loc(user_id, df = users_df):   
    ''' this finction grabs the location of an user from ahoy_events dataset'''
    
    users_idx = users_df[users_df['id']== user_id]
    # now we get the user's latitude and longitude
    
    location_dict = {'user_latitude' : float(users_idx.latitude.values), 
                     
                     'user_longitude' : float(users_idx.longitude.values)
                        }
     
    return location_dict


def content_recommender(user_id, cosine_sim = cosine_sim, df = events_df, title_to_index = title_to_index):
    '''This recommender finds out 20 closest matches with a given event'''
    
    #calling user id from ahoy_events
    
    title = get_title(user_id, df = ahoy_events_df)
    
    idx = title_to_index[title]
    
    sim_scores = list(enumerate(cosine_sim[idx].todense().tolist()[0]))
    # list of set [(index, sim score), (index, sim score), ...]
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Sort the list of set by sim score, index with highest simlirarity
    # Will be at the beginning of the list
    
    sim_scores = sim_scores[1:101] # top 100 events (first one is our input events so we ignore it)
    
    sport_indices = [i[0] for i in sim_scores] # we grab the indices of those 10 events
    
    df_all_recommendations = df.iloc[sport_indices][[ 'id','title', 'latitude', 'longitude', 'offer_date']]
    
    df_date_filter = df_all_recommendations[df_all_recommendations['offer_date'] >= datetime.date.today()]
    
    #calcullation of haversine distance for users:
    user_latitude = get_user_loc(user_id, df = users_df)['user_latitude']
    user_longitude = get_user_loc(user_id, df = users_df)['user_longitude']
    
    df_date_filter['user_latitude'] = user_latitude
    df_date_filter['user_longitude'] = user_longitude
    
    df_date_filter["distance"] = haversine_vectorized(df_date_filter.latitude, df_date_filter.longitude,
                                                      df_date_filter.user_latitude, df_date_filter.user_longitude)
            
    df_location_filter = df_date_filter[df_date_filter['distance'] <= 200.0]
    
    
                                                
    return df_location_filter.title
#     return user_latitude

In [ ]:
ahoy_events_df.user_id.unique()

In [ ]:
content_recommender(65)

# updating event prediction: finalizing stuff

In [ ]:
def get_event_id(user_id):
    ''' this finction grabs the title of an event used by an user from ahoy_events dataset'''
    
    event_id = ahoy_events_df[ahoy_events_df['user_id']== user_id].iloc[-1].properties['offer']
    # now we get the title of the event
    #event_id = events_df[events_df['id']==event_idx].title
    
    return event_id 



In [ ]:
ahoy_events_df.user_id.unique()

In [ ]:
get_event_id(65)

In [ ]:
events_df.columns

In [ ]:
events_df[events_df['id']== get_event_id(65)].index



In [ ]:
#id_to_index = pd.Series(events_df.id, index = events_df[get_event_id(65)]).drop_duplicates()

In [ ]:
id_to_index

In [ ]:
ahoy_events_df.user_id

In [ ]:
ahoy_events_df.head()

In [ ]:
users_df.head()

In [ ]:
from datetime import date

def get_event_id(user_id, df = ahoy_events_df):
    ''' this finction grabs the title of an event used by an user from ahoy_events dataset'''
    
    event_id = ahoy_events_df[ahoy_events_df['user_id']== user_id].iloc[0].properties['offer']
    # now we get the title of the event
    #event_id = events_df[events_df['id']==event_idx].title
    
    return event_id 

def get_user_loc(user_id, df = users_df):   
    ''' this finction grabs the location of an user from ahoy_events dataset'''
    
    users_idx = users_df[users_df['id']== user_id]
    # now we get the user's latitude and longitude
    
    location_dict = {'user_latitude' : float(users_idx.latitude.values), 
                     
                     'user_longitude' : float(users_idx.longitude.values)
                        }
     
    return location_dict


def content_recommender(user_id, cosine_sim = cosine_sim, df = events_df):
    '''This recommender finds out 100 closest matches with a given event'''
    
    #calling user id from ahoy_events
    
    event_id = get_event_id(user_id, df = ahoy_events_df)
    
    event_idx = events_df[events_df['id']== get_event_id(user_id)].index

    
    sim_scores = list(enumerate(cosine_sim[event_idx].todense().tolist()[0]))
    # list of set [(index, sim score), (index, sim score), ...]
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Sort the list of set by sim score, index with highest simlirarity
    # Will be at the beginning of the list
    
    sim_scores = sim_scores[1:101] # top 100 events (first one is our input events so we ignore it)
    
    sport_indices = [i[0] for i in sim_scores] # we grab the indices of those 10 events
    
    df_all_recommendations = df.iloc[sport_indices][[ 'id','title', 'latitude', 'longitude', 'offer_date']]
    
    df_date_filter = df_all_recommendations[df_all_recommendations['offer_date'] >= datetime.date.today()]
    
    #calcullation of haversine distance for users:
    user_latitude = get_user_loc(user_id, df = users_df)['user_latitude']
    user_longitude = get_user_loc(user_id, df = users_df)['user_longitude']
    
    df_date_filter['user_latitude'] = user_latitude
    df_date_filter['user_longitude'] = user_longitude
    
    df_date_filter["distance"] = haversine_vectorized(df_date_filter.latitude, df_date_filter.longitude,
                                                      df_date_filter.user_latitude, df_date_filter.user_longitude)
            
    df_location_filter = df_date_filter[df_date_filter['distance'] <= 200.0]
    
    
                                                
    return df_location_filter.title
#     return user_latitude

In [ ]:
content_recommender(65)